In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from time import time
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import psutil
import os
def get_mem_usage():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0]/2.**30
    return memoryUse

In [3]:
from utils import reduce_mem_usage, ClassifierTransformer, MoreStructureProperties, MakeMoreFeatures

In [4]:
def map_atom_info(df_1, df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    return df

    
def find_dist(df):
    df_p_0 = df[['x_0', 'y_0', 'z_0']].values
    df_p_1 = df[['x_1', 'y_1', 'z_1']].values
    
    df['dist'] = np.linalg.norm(df_p_0 - df_p_1, axis=1)
    df['dist_inv2'] = 1/df['dist']**2
    df['dist_x'] = (df['x_0'] - df['x_1']) ** 2
    df['dist_y'] = (df['y_0'] - df['y_1']) ** 2
    df['dist_z'] = (df['z_0'] - df['z_1']) ** 2
    return df

def find_closest_atom(df):    
    df_temp = df.loc[:,["molecule_name",
                      "atom_index_0","atom_index_1",
                      "dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_ = df_temp.copy()
    df_temp_ = df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp_all = pd.concat((df_temp,df_temp_),axis=0)

    df_temp_all["min_distance"]=df_temp_all.groupby(['molecule_name', 
                                                     'atom_index_0'])['dist'].transform('min')
    df_temp_all["max_distance"]=df_temp_all.groupby(['molecule_name', 
                                                     'atom_index_0'])['dist'].transform('max')
    
    df_temp = df_temp_all[df_temp_all["min_distance"]==df_temp_all["dist"]].copy()
    df_temp = df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp = df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_closest',
                                         'dist': 'distance_closest',
                                         'x_1': 'x_closest',
                                         'y_1': 'y_closest',
                                         'z_1': 'z_closest'})
    df_temp = df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])
    
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'distance_closest': f'distance_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})
        
    df_temp= df_temp_all[df_temp_all["max_distance"]==df_temp_all["dist"]].copy()
    df_temp = df_temp.drop(['x_0','y_0','z_0','max_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_farthest',
                                         'dist': 'distance_farthest',
                                         'x_1': 'x_farthest',
                                         'y_1': 'y_farthest',
                                         'z_1': 'z_farthest'})
    df_temp = df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])
        
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                                        'distance_farthest': f'distance_farthest_{atom_idx}',
                                        'x_farthest': f'x_farthest_{atom_idx}',
                                        'y_farthest': f'y_farthest_{atom_idx}',
                                        'z_farthest': f'z_farthest_{atom_idx}'})
    return df


def add_cos_features(df):
    
    df["distance_center0"] = np.sqrt((df['x_0']-df['c_x'])**2 \
                                   + (df['y_0']-df['c_y'])**2 \
                                   + (df['z_0']-df['c_z'])**2)
    df["distance_center1"] = np.sqrt((df['x_1']-df['c_x'])**2 \
                                   + (df['y_1']-df['c_y'])**2 \
                                   + (df['z_1']-df['c_z'])**2)
    
    df['distance_c0'] = np.sqrt((df['x_0']-df['x_closest_0'])**2 + \
                                (df['y_0']-df['y_closest_0'])**2 + \
                                (df['z_0']-df['z_closest_0'])**2)
    df['distance_c1'] = np.sqrt((df['x_1']-df['x_closest_1'])**2 + \
                                (df['y_1']-df['y_closest_1'])**2 + \
                                (df['z_1']-df['z_closest_1'])**2)
    
    df["distance_f0"] = np.sqrt((df['x_0']-df['x_farthest_0'])**2 + \
                                (df['y_0']-df['y_farthest_0'])**2 + \
                                (df['z_0']-df['z_farthest_0'])**2)
    df["distance_f1"] = np.sqrt((df['x_1']-df['x_farthest_1'])**2 + \
                                (df['y_1']-df['y_farthest_1'])**2 + \
                                (df['z_1']-df['z_farthest_1'])**2)
    
    vec_center0_x = (df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
    vec_center0_y = (df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
    vec_center0_z = (df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
    
    vec_center1_x = (df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
    vec_center1_y = (df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
    vec_center1_z = (df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
    
    vec_c0_x = (df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
    vec_c0_y = (df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
    vec_c0_z = (df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
    
    vec_c1_x = (df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
    vec_c1_y = (df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
    vec_c1_z = (df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
    
    vec_f0_x = (df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
    vec_f0_y = (df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
    vec_f0_z = (df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
    
    vec_f1_x = (df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
    vec_f1_y = (df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
    vec_f1_z = (df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
    
    vec_x = (df['x_1']-df['x_0'])/df['dist']
    vec_y = (df['y_1']-df['y_0'])/df['dist']
    vec_z = (df['z_1']-df['z_0'])/df['dist']
    
    df["cos_c0_c1"] = vec_c0_x*vec_c1_x + vec_c0_y*vec_c1_y + vec_c0_z*vec_c1_z
    df["cos_f0_f1"] = vec_f0_x*vec_f1_x + vec_f0_y*vec_f1_y + vec_f0_z*vec_f1_z
    
    df["cos_c0_f0"] = vec_c0_x*vec_f0_x + vec_c0_y*vec_f0_y + vec_c0_z*vec_f0_z
    df["cos_c1_f1"] = vec_c1_x*vec_f1_x + vec_c1_y*vec_f1_y + vec_c1_z*vec_f1_z
    
    df["cos_center0_center1"] = vec_center0_x*vec_center1_x \
                              + vec_center0_y*vec_center1_y \
                              + vec_center0_z*vec_center1_z
    
    df["cos_c0"] = vec_c0_x*vec_x + vec_c0_y*vec_y + vec_c0_z*vec_z
    df["cos_c1"] = vec_c1_x*vec_x + vec_c1_y*vec_y + vec_c1_z*vec_z
    
    df["cos_f0"] = vec_f0_x*vec_x + vec_f0_y*vec_y + vec_f0_z*vec_z
    df["cos_f1"] = vec_f1_x*vec_x + vec_f1_y*vec_y + vec_f1_z*vec_z
    
    df["cos_center0"] = vec_center0_x*vec_x + vec_center0_y*vec_y + vec_center0_z*vec_z
    df["cos_center1"] = vec_center1_x*vec_x + vec_center1_y*vec_y + vec_center1_z*vec_z

    return df

def dummies(df, list_cols):
    for col in list_cols:
        df_dummies = pd.get_dummies(df[col], drop_first=True, 
                                    prefix=(str(col)))
        df = pd.concat([df, df_dummies], axis=1)
    return df

In [5]:
def add_qm9_features(df):
    data_qm9 = pd.read_pickle('../data/raw/data.covs.pickle')
    to_drop = ['type', 
               'linear', 
               'atom_index_0', 
               'atom_index_1', 
               'scalar_coupling_constant', 
               'U', 'G', 'H', 
               'mulliken_mean', 'r2', 'U0']
    data_qm9 = data_qm9.drop(columns = to_drop, axis=1)
    data_qm9 = reduce_mem_usage(data_qm9,verbose=False)
    df = pd.merge(df, data_qm9, how='left', on=['molecule_name','id'])
    del data_qm9
    
    df = dummies(df, ['type', 'atom_1'])
    return df

def get_features(df, struct):
    for atom_idx in [0,1]:
        df = map_atom_info(df, struct, atom_idx)
        df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
        struct['c_x'] = struct.groupby('molecule_name')['x'].transform('mean')
        struct['c_y'] = struct.groupby('molecule_name')['y'].transform('mean')
        struct['c_z'] = struct.groupby('molecule_name')['z'].transform('mean')

    df = find_dist(df)
    df = find_closest_atom(df)
    df = add_cos_features(df)
    df = add_qm9_features(df)
    return df

In [6]:
def feat_from_structures(df, st):
    df = pd.merge(df,st,how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'])
    df = pd.merge(df,st,how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'])
    n_atoms = st.groupby(['molecule_name','atom'])['atom'].size().to_frame(name = 'count').reset_index()
    n_atoms_df = n_atoms.pivot_table('count',['molecule_name'], 'atom')
    n_atoms_df.fillna(0,inplace=True)
    df = pd.merge(df,n_atoms_df,on=['molecule_name'],how='left')
    del n_atoms
    gc.collect()
    return df

In [7]:
def comp_score (y_true, y_pred, jtype):
    df = pd.DataFrame()
    df['y_true'] , df['y_pred'], df['jtype'] = y_true , y_pred, jtype
    score = 0 
    for t in jtype.unique():
        score_jtype = np.log(mean_absolute_error(df[df.jtype==t]['y_true'],df[df.jtype==t]['y_pred']))
        score += score_jtype
        print(f'{t} : {score_jtype}')
    score /= len(jtype.unique())
    return score

In [8]:
def feat_from_structures(df, st):
    df = pd.merge(df,st,how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'])
    df = pd.merge(df,st,how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'])
    n_atoms = st.groupby(['molecule_name','atom'])['atom'].size().to_frame(name = 'count').reset_index()
    n_atoms_df = n_atoms.pivot_table('count',['molecule_name'], 'atom')
    n_atoms_df.fillna(0,inplace=True)
    df = pd.merge(df,n_atoms_df,on=['molecule_name'],how='left')
    del n_atoms
    gc.collect()
    return df


In [9]:
atomic_radius = {'H': 0.43, 'C': 0.82, 'N': 0.8, 'O': 0.78, 'F': 0.76}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}

In [10]:
pipeline_model1 = make_pipeline(MoreStructureProperties(atomic_radius,electronegativity))
pipeline_model2 = make_pipeline(MakeMoreFeatures())

train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')
struct = pd.read_csv('../data/raw/structures.csv')
structures_yukawa = pd.read_csv('../data/raw/structures_yukawa.csv')

struct = pd.concat([struct, structures_yukawa], axis=1)
del structures_yukawa

struct = reduce_mem_usage(struct,verbose=False)
gc.collect()

train = get_features(train, struct.copy())
test = get_features(test, struct.copy())
y = train['scalar_coupling_constant']

del struct
gc.collect()

161

In [11]:
struct = pd.read_csv('../data/raw/structures.csv')
struct = pipeline_model1.fit_transform(struct)

train = feat_from_structures(train,struct)
train = pipeline_model2.fit_transform(
    train.drop(['scalar_coupling_constant'], axis=1),
    train['scalar_coupling_constant']
)

test = feat_from_structures(test,struct)
test = pipeline_model2.transform(test)

train = reduce_mem_usage(train,verbose=False)
test = reduce_mem_usage(test,verbose=False)

/root/CHAMPS/experiments/utils.py:184: RuntimeWarning: Mean of empty slice
  X['dist_mean_no_bond'] = np.nanmean(np.where(dist_matrix_no_bond==0,np.nan,dist_matrix), axis=1)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)
/root/CHAMPS/experiments/utils.py:194: RuntimeWarning: All-NaN slice encountered
  X['dist_no_bond_min'] = np.nanmin(np.where(dist_matrix_no_bond==0,np.nan,dist_matrix), axis=1)
/root/CHAMPS/experiments/utils.py:195: RuntimeWarning: All-NaN slice encountered
  X['dist_no_bond_max'] = np.nanmax(np.where(dist_matrix_no_bond==0,np.nan,dist_matrix), axis=1)


In [12]:
pd.__version__

'0.24.2'

In [12]:
giba_columns = ['inv_dist0', 'inv_dist1', 'inv_distP', 'inv_dist0R', 'inv_dist1R', 'inv_distPR', 'inv_dist0E', 'inv_dist1E', 'inv_distPE', 'linkM0',
         'linkM1', 'min_molecule_atom_0_dist_xyz', 'mean_molecule_atom_0_dist_xyz', 'max_molecule_atom_0_dist_xyz', 'sd_molecule_atom_0_dist_xyz', 'min_molecule_atom_1_dist_xyz',
         'mean_molecule_atom_1_dist_xyz', 'max_molecule_atom_1_dist_xyz', 'sd_molecule_atom_1_dist_xyz', 'coulomb_C.x', 'coulomb_F.x', 'coulomb_H.x', 'coulomb_N.x',
         'coulomb_O.x', 'yukawa_C.x', 'yukawa_F.x', 'yukawa_H.x', 'yukawa_N.x', 'yukawa_O.x', 'vander_C.x', 'vander_F.x', 'vander_H.x', 'vander_N.x', 'vander_O.x',
         'coulomb_C.y', 'coulomb_F.y', 'coulomb_H.y', 'coulomb_N.y', 'coulomb_O.y', 'yukawa_C.y', 'yukawa_F.y', 'yukawa_H.y', 'yukawa_N.y', 'yukawa_O.y', 'vander_C.y',
         'vander_F.y', 'vander_H.y', 'vander_N.y', 'vander_O.y', 'distC0', 'distH0', 'distN0', 'distC1', 'distH1', 'distN1', 'adH1', 'adH2', 'adH3', 'adH4', 'adC1',
         'adC2', 'adC3', 'adC4', 'adN1', 'adN2', 'adN3', 'adN4', 'NC', 'NH', 'NN', 'NF', 'NO']

In [13]:
train_giba_t = pd.read_csv('../data/raw/train_giba.csv.gz',
                        header=0,  usecols=giba_columns)
test_giba_t = pd.read_csv('../data/raw/test_giba.csv.gz',
                       header=0,  usecols=giba_columns)
train_giba_t = reduce_mem_usage(train_giba_t)
test_giba_t = reduce_mem_usage(test_giba_t)

train = pd.concat((train,train_giba_t),axis=1)
test = pd.concat((test,test_giba_t),axis=1)

Mem. usage decreased to 635.26 Mb (75.2% reduction)
Mem. usage decreased to 339.30 Mb (75.3% reduction)


In [14]:
del train_giba_t
del test_giba_t
gc.collect()

69

In [15]:
all_features = ['type',   'x_x', 'y_x','z_x', 'atom_y', 'x_y', 'y_y',
       'z_y', 'n_bonds_y', 'C', 'F', 'H', 'N', 'O', 'distance', 'dist_mean_x','dist_mean_y',
        'x_dist_abs', 'y_dist_abs', 'z_dist_abs','inv_distance3',
       'molecule_atom_1_dist_std_diff','molecule_dist_mean_x',
       'molecule_dist_mean_y','molecule_dist_std_x','molecule_dist_std_y','molecule_atom_0_dist_mean',
       'molecule_atom_1_dist_mean','dist_mean_bond_y',
       'n_no_bonds_x','n_no_bonds_y', 'dist_std_x', 'dist_std_y','dist_min_x','dist_min_y','dist_max_x', 'dist_max_y',
       'molecule_dist_range_x','molecule_dist_range_y', 'dimension_x', 'dimension_y','dimension_z','molecule_dist_mean_bond_x',
       'molecule_dist_mean_bond_x','dist_mean_no_bond_x','dist_mean_no_bond_y',
       'dist_std_bond_y','dist_bond_min_y','dist_bond_max_y',
       'range_dist_bond_y','dist_std_no_bond_x','dist_std_no_bond_y', 'dist_no_bond_min_x','dist_no_bond_min_y','dist_no_bond_max_x',
       'dist_no_bond_max_y', 'range_dist_no_bond_x','range_dist_no_bond_y','dist_median_bond_y','dist_median_x',
       'dist_median_y','dist_median_no_bond_x','dist_median_no_bond_y','molecule_type_dist_min','molecule_type_dist_max',
       'molecule_dist_mean_no_bond_x','molecule_dist_mean_no_bond_y', 'n_diff_y','molecule_atom_index_0_dist_min_div','molecule_atom_index_0_dist_std_div',
        'molecule_atom_index_0_dist_mean','molecule_atom_index_0_dist_max','molecule_atom_index_1_dist_mean','molecule_atom_index_1_dist_max',
       'molecule_atom_index_1_dist_min','molecule_atom_index_1_dist_min_div','molecule_atom_index_1_dist_std_diff','molecule_atom_index_0_dist_mean_diff',
        'molecule_atom_index_1_dist_mean_div','molecule_atom_index_1_dist_min_diff', 'rc_A', 'rc_B', 'rc_C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'zpve', 'Cv',
         'freqs_min', 'freqs_max', 'freqs_mean', 'mulliken_min', 'mulliken_max', 'mulliken_atom_0', 'mulliken_atom_1',
         'dist_C_0_x', 'dist_C_1_x', 'dist_C_2_x', 'dist_C_3_x', 'dist_C_4_x', 'dist_F_0_x', 'dist_F_1_x', 'dist_F_2_x', 'dist_H_0_x',
         'dist_H_1_x', 'dist_H_2_x', 'dist_H_3_x', 'dist_H_4_x', 'dist_N_0_x', 'dist_N_1_x', 'dist_N_2_x', 'dist_N_3_x', 'dist_N_4_x', 'dist_O_0_x', 'dist_O_1_x',
         'dist_O_2_x', 'dist_O_3_x', 'dist_O_4_x', 'dist_C_0_y', 'dist_C_1_y', 'dist_C_2_y', 'dist_C_3_y', 'dist_C_4_y', 'dist_F_0_y', 'dist_F_1_y', 'dist_F_2_y',
         'dist_F_3_y', 'dist_F_4_y', 'dist_H_0_y', 'dist_H_1_y', 'dist_H_2_y', 'dist_H_3_y', 'dist_H_4_y', 'dist_N_0_y', 'dist_N_1_y', 'dist_N_2_y', 'dist_N_3_y',
         'dist_N_4_y', 'dist_O_0_y', 'dist_O_1_y', 'dist_O_2_y', 'dist_O_3_y', 'dist_O_4_y','distance_closest_0', 'distance_closest_1', 'distance_farthest_0',
         'distance_farthest_1','cos_c0_c1', 'cos_f0_f1','cos_c0_f0', 'cos_c1_f1', 'cos_center0_center1', 'cos_c0', 'cos_c1', 'cos_f0', 'cos_f1',
         'cos_center0', 'cos_center1'] + giba_columns

In [16]:
cat_features = ['atom_y']

In [17]:
from sklearn.model_selection import KFold
X = train[all_features]

n_folds = 10

seed = 42

params = {
    'num_leaves': 50,
    'min_child_samples': 79,
    'min_data_in_leaf': 100,
    'objective': 'regression',
    'max_depth': 9,
    'learning_rate': 0.2,
    "boosting_type": "gbdt",
    "subsample_freq": 1,
    "subsample": 0.9,
    "bagging_seed": 11,
    "metric": 'mae',
    "verbosity": -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0,
    'num_iterations': 20000
}

folds = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

metaparams = {
    'rf_n_estimators': 100,
    'importance_num_iterations': 1000
}

In [18]:
from utils import fast_metric

In [19]:
X_test = test[all_features]
sub = pd.DataFrame()
sub['id'] = test['id']
sub['type'] = test['type']
pred_sub = np.zeros(sub.shape[0])
all_features.pop(0)
unique_types = train['type'].unique()

In [20]:
import datetime

In [21]:
LOG_FILENAME = f"LGB_log_{str(datetime.date.today())}-{str(datetime.datetime.now().hour)}.txt"
def logprint(*args):
    logstring = f"[{str(datetime.datetime.now())}] - " + (" ".join(list(args)))
    with open(LOG_FILENAME, 'a+', encoding='utf-8') as file:
        file.write(logstring + "\n")
        
    print(logstring)
        

In [ ]:
scores = []

i = 1

logprint("Beginning the training")
logprint(f"# folds: {n_folds}")
logprint(f"metaparams: {metaparams}")
logprint(f"lgb params: {params}")
for i_train, i_val in folds.split(X):
    logprint("-"*50)

    logprint(f"Training for split {i}")
    i += 1
    
    X_train, X_val, y_train, y_val = X.iloc[i_train], X.iloc[i_val], y[i_train], y[i_val]
    pred = np.zeros(X_val.shape[0])


    d_map_val = dict(zip(list(y_val.index.values),list(np.arange(y_val.shape[0]))))

    rf_cols = ['rf_00','rf_01','rf_02','rf_03','rf04','rf05']
    rf_cols1 = ['rf_10','rf_11','rf12']
    for t in unique_types:
        logprint(f"     Type: {t}")
        
        evals_result = {}
        
        idx_train = X_train[X_train.type==t].index.values
        idx_val = X_val[X_val.type==t].index.values
        idx_sub = sub[sub.type==t].index.values
        
        clf = ClassifierTransformer(RandomForestClassifier(n_estimators=metaparams['rf_n_estimators'], n_jobs=28),n_classes=5,cv=5)
        clf1 = ClassifierTransformer(RandomForestClassifier(n_estimators=metaparams['rf_n_estimators'], n_jobs=28),n_classes=2,cv=5)
       
        logprint("     training and applying feature classifiers")
        logprint(f"     memory footprint:{get_mem_usage():.4f} GB")
        X_extra = np.hstack([clf.fit_transform(X_train[X_train.type==t].drop(['type'],axis=1), y_train[idx_train]),clf1.fit_transform(X_train[X_train.type==t].drop(['type'],axis=1), y_train[idx_train])])
        X_extra_val = np.hstack([clf.transform(X_val[X_val.type==t].drop(['type'],axis=1)),clf1.transform(X_val[X_val.type==t].drop(['type'],axis=1))])
        X_extra_test = np.hstack([clf.transform(X_test[X_test.type==t].drop(['type'],axis=1)),clf1.transform(X_test[X_test.type==t].drop(['type'],axis=1))])
        
        X_p = pd.DataFrame(data=np.hstack([X_train[X_train.type==t].drop(['type'],axis=1).values,X_extra]), columns=list(X_train.drop(['type'],axis=1).columns)+rf_cols+rf_cols1)
        X_p_val = pd.DataFrame(data=np.hstack([X_val[X_val.type==t].drop(['type'],axis=1).values,X_extra_val]), columns=list(X_val.drop(['type'],axis=1).columns)+rf_cols+rf_cols1)
        X_p_test = pd.DataFrame(data=np.hstack([X_test[X_test.type==t].drop(['type'],axis=1).values,X_extra_test]), columns=list(X_test.drop(['type'],axis=1).columns)+rf_cols+rf_cols1)

        logprint("     training feature importance estimator")
        logprint(f"     memory footprint:{get_mem_usage():.4f} GB")

        gbm = lgb.LGBMRegressor(n_estimators=metaparams['importance_num_iterations'])
        gbm.fit(X_p, y_train[idx_train])
        gbm.booster_.feature_importance()

        fea_imp_ = pd.DataFrame({'cols':X_p.columns, 'fea_imp':gbm.feature_importances_})
        
        if t=='1JHC':
            nb_feat=100
        else:
            nb_feat=90
            
        remain_features = list(fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)['cols'].values[:nb_feat])
        logprint(f"    Top20 features: {remain_features[:20]}")

        logprint(f'    Training final estiamtor for {t}')
        lgb_train = lgb.Dataset(X_p[remain_features],label=y_train[idx_train])
        lgb_val = lgb.Dataset(X_p_val[remain_features],label=y_val[idx_val])
        model = lgb.train(params=params, train_set=lgb_train,  valid_sets=[lgb_train,lgb_val], valid_names=['train','val'],
                      fobj=None, feval=None, init_model=None, feature_name='auto', categorical_feature='auto',evals_result=evals_result,
                      early_stopping_rounds=200,  verbose_eval=500)
        logprint("     Training finished, predicting something")
        val_map = [d_map_val[k] for k in list(idx_val)]
        pred[val_map] = model.predict(X_p_val[remain_features])
        pred_sub[idx_sub] += (model.predict(X_p_test[remain_features]) / n_folds)
        logprint(f"     memory footprint:{get_mem_usage():.4f} GB")

        logprint("     reducing amount of garbadge")
        del X_p
        del X_p_val
        del X_p_test
        del X_extra
        del X_extra_val
        del X_extra_test
        del clf
        del clf1
        del gbm
        gc.collect()
        logprint("     garbadge eliminated")
        logprint(f"     memory footprint:{get_mem_usage():.4f} GB")


    jtype = X_val['type']
    scores.append(fast_metric(y_val, pred, jtype, verbose=False))
    logprint(f"Fold result: {scores[-1]:.4f} group-mean log(mae)")

CV_score = np.mean(scores)
CV_std = np.std(scores)

logprint(f"Cross-Validation score over {n_folds} folds: {CV_score:.4f} +- {CV_std:.4f}")

[2019-07-26 10:28:00.934841] - Beginning the training
[2019-07-26 10:28:00.935019] - # folds: 5
[2019-07-26 10:28:00.935159] - metaparams: {'rf_n_estimators': 10, 'importance_num_iterations': 1000}
[2019-07-26 10:28:00.935752] - lgb params: {'num_leaves': 50, 'min_child_samples': 79, 'min_data_in_leaf': 100, 'objective': 'regression', 'max_depth': 9, 'learning_rate': 0.2, 'boosting_type': 'gbdt', 'subsample_freq': 1, 'subsample': 0.9, 'bagging_seed': 11, 'metric': 'mae', 'verbosity': -1, 'reg_alpha': 0.1, 'reg_lambda': 0.3, 'colsample_bytree': 1.0, 'num_iterations': 10000}
[2019-07-26 10:28:01.122732] - --------------------------------------------------
[2019-07-26 10:28:01.122920] - Training for split 1
[2019-07-26 10:28:04.002383] -      Type: 1JHC
[2019-07-26 10:28:04.739998] -      training and applying feature classifiers
[2019-07-26 10:28:04.740602] -      memory footprint:11.8243 GB
[2019-07-26 10:37:15.904456] -      training feature importance estimator
[2019-07-26 10:37:15.90

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[500]	train's l1: 0.838954	val's l1: 0.948793
[1000]	train's l1: 0.683237	val's l1: 0.846056
[1500]	train's l1: 0.589249	val's l1: 0.792758
[2000]	train's l1: 0.52253	val's l1: 0.760159
[2500]	train's l1: 0.470108	val's l1: 0.737563
[3000]	train's l1: 0.427097	val's l1: 0.720795
[3500]	train's l1: 0.391529	val's l1: 0.707892
[4000]	train's l1: 0.360967	val's l1: 0.697235
[4500]	train's l1: 0.334118	val's l1: 0.688702
[5000]	train's l1: 0.31038	val's l1: 0.681913
[5500]	train's l1: 0.289086	val's l1: 0.675626
[6000]	train's l1: 0.270068	val's l1: 0.6708
[6500]	train's l1: 0.252888	val's l1: 0.66644
[7000]	train's l1: 0.236976	val's l1: 0.662532
[7500]	train's l1: 0.2228	val's l1: 0.659584
[8000]	train's l1: 0.209562	val's l1: 0.656577
[8500]	train's l1: 0.197455	val's l1: 0.654191
[9000]	train's l1: 0.186178	val's l1: 0.652033
[9500]	train's l1: 0.175806	val's l1: 0.650009
[10000]	train's l1: 0.166192	val's l1: 0.648338
Did 

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[500]	train's l1: 0.156575	val's l1: 0.194732
[1000]	train's l1: 0.120087	val's l1: 0.174298
[1500]	train's l1: 0.0993682	val's l1: 0.165269
[2000]	train's l1: 0.0848221	val's l1: 0.160086
[2500]	train's l1: 0.0739003	val's l1: 0.15678
[3000]	train's l1: 0.0650491	val's l1: 0.154417
[3500]	train's l1: 0.0578219	val's l1: 0.152801
[4000]	train's l1: 0.0517264	val's l1: 0.151571
[4500]	train's l1: 0.0464849	val's l1: 0.150601
[5000]	train's l1: 0.0420132	val's l1: 0.149922
[5500]	train's l1: 0.0380624	val's l1: 0.149293
[6000]	train's l1: 0.0345967	val's l1: 0.148764
[6500]	train's l1: 0.03153	val's l1: 0.148367
[7000]	train's l1: 0.0287826	val's l1: 0.14804
[7500]	train's l1: 0.0263498	val's l1: 0.147776
[8000]	train's l1: 0.0241221	val's l1: 0.147536
[8500]	train's l1: 0.022137	val's l1: 0.147355
[9000]	train's l1: 0.0203466	val's l1: 0.147206
[9500]	train's l1: 0.0187314	val's l1: 0.147077
[10000]	train's l1: 0.0172575	val

...

Training until validation scores don't improve for 200 rounds.
[500]	train's l1: 0.124347	val's l1: 0.206747
[1000]	train's l1: 0.0802683	val's l1: 0.192741
[1500]	train's l1: 0.0564339	val's l1: 0.187686
[2000]	train's l1: 0.0414845	val's l1: 0.185235
[2500]	train's l1: 0.0311858	val's l1: 0.18386
[3000]	train's l1: 0.0238572	val's l1: 0.183071
[3500]	train's l1: 0.0185156	val's l1: 0.182569
[4000]	train's l1: 0.0145786	val's l1: 0.182216
[4500]	train's l1: 0.0115565	val's l1: 0.181951
[5000]	train's l1: 0.00929246	val's l1: 0.181769
[5500]	train's l1: 0.00753474	val's l1: 0.181642
[6000]	train's l1: 0.00619855	val's l1: 0.181558
[6500]	train's l1: 0.00516216	val's l1: 0.181494
[7000]	train's l1: 0.00434629	val's l1: 0.18144
[7500]	train's l1: 0.00370038	val's l1: 0.181406
[8000]	train's l1: 0.00319108	val's l1: 0.181376
[8500]	train's l1: 0.00277844	val's l1: 0.181353
[9000]	train's l1: 0.00244672	val's l1: 0.181336
[9500]	train's l1: 0.0021754	val's l1: 0.181325
[10000]	train's l1: 

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[500]	train's l1: 0.400751	val's l1: 0.43786
[1000]	train's l1: 0.338171	val's l1: 0.39309
[1500]	train's l1: 0.299732	val's l1: 0.369182
[2000]	train's l1: 0.271743	val's l1: 0.35349
[2500]	train's l1: 0.250055	val's l1: 0.342534
[3000]	train's l1: 0.23183	val's l1: 0.333919
[3500]	train's l1: 0.216442	val's l1: 0.327025
[4000]	train's l1: 0.203165	val's l1: 0.321636
[4500]	train's l1: 0.191462	val's l1: 0.317079
[5000]	train's l1: 0.181078	val's l1: 0.313364
[5500]	train's l1: 0.171586	val's l1: 0.309991
[6000]	train's l1: 0.16304	val's l1: 0.307104
[6500]	train's l1: 0.155242	val's l1: 0.304625
[7000]	train's l1: 0.148083	val's l1: 0.302367
[7500]	train's l1: 0.141443	val's l1: 0.300427
[8000]	train's l1: 0.13526	val's l1: 0.29861
[8500]	train's l1: 0.129542	val's l1: 0.297138
[9000]	train's l1: 0.124186	val's l1: 0.295758
[9500]	train's l1: 0.119117	val's l1: 0.294478
[10000]	train's l1: 0.114416	val's l1: 0.293316
Did 

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[500]	train's l1: 0.206512	val's l1: 0.23793
[1000]	train's l1: 0.164814	val's l1: 0.211052
[1500]	train's l1: 0.140339	val's l1: 0.197619
[2000]	train's l1: 0.122901	val's l1: 0.189242
[2500]	train's l1: 0.109739	val's l1: 0.183638
[3000]	train's l1: 0.0989947	val's l1: 0.179705
[3500]	train's l1: 0.0900969	val's l1: 0.176693
[4000]	train's l1: 0.0825075	val's l1: 0.17432
[4500]	train's l1: 0.0758521	val's l1: 0.17241
[5000]	train's l1: 0.0700442	val's l1: 0.170818
[5500]	train's l1: 0.0648568	val's l1: 0.169574
[6000]	train's l1: 0.060252	val's l1: 0.168527
[6500]	train's l1: 0.0561242	val's l1: 0.167587
[7000]	train's l1: 0.0523799	val's l1: 0.166817
[7500]	train's l1: 0.0489794	val's l1: 0.166158
[8000]	train's l1: 0.0458351	val's l1: 0.165584
[8500]	train's l1: 0.043009	val's l1: 0.165079
[9000]	train's l1: 0.0404082	val's l1: 0.164648
[9500]	train's l1: 0.037985	val's l1: 0.164233
[10000]	train's l1: 0.0357632	val's l

In [ ]:
sub['scalar_coupling_constant'] = pred_sub
sub[['id','scalar_coupling_constant']].to_csv(f'../submits/LGB_CV_[{n_folds}]_[{CV_score:.4f}+-{CV_std:.4f}].csv',index=False)